# Code Setup

In [59]:
import numpy as np
import itertools
from functools import partial
from typing import Callable, Iterator

from utils.import_puzzle_input import load_input
from utils.import_puzzle_input import split_puzzle_input

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload modules before executing code
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
puzzle_input = load_input(2023, 12)

loaded puzzle input


In [61]:
puzzle_input_split = split_puzzle_input(puzzle_input)
puzzle_input_split[0:20]

['??????#?#?#?? 2,2,6',
 '#?????????#???? 1,3,2,2',
 '?#??#..#?#???#??#. 5,1,1,4',
 '#??????#???##??#???? 2,1,3,9',
 '??.?????.? 3,1',
 '?.?.?#?##????.?.?# 1,1,3,2,1',
 '#..?????#.???#? 1,1,1,1,4',
 '#?#??????#??????. 1,1,8,1,1',
 '??????.????#?#??. 2,1,6',
 '?#?????.?..??.#??. 1,2,1,2,1,1',
 '??.#?.#???.#??? 1,2,1,1,4',
 '###?.?#??.?#?. 4,4,2',
 '.?...??#??? 1,1',
 '????#.?.???#??#?#?. 2,2,3,5',
 '.??.?#????????? 1,2,1,1,1',
 '?#.????#????.????# 1,6,1,2',
 '?#???.?????### 2,1,3,4',
 '?????#.??# 2,1,3',
 '??????.?##?#??#????? 3,1,12',
 '.?..??.????#?? 1,1,3,1']

In [62]:
# # censored condition records
# pis_ccr = [line.split(" ")[0] for line in puzzle_input_split]

# #puzzle_inout_split check
# pis_check = [line.split(" ")[1] for line in puzzle_input_split]

pis_ccr, pis_wrs = map(list, zip(*[line.split(" ") for line in puzzle_input_split]))
# puzzle_input_split_ccr, puzzle_input_split_wrs = map(list, zip(*[line.split(" ") for line in puzzle_input_split]))

In [63]:
print(pis_ccr[0:20])
# # Let's make a function for trimming the beginning and end of lines for "."s
# we use "in {}" speedup as benchmarked to be faster than "== or ==" or "in []"
# !!! 25-06-08: Should be able to speed up more by having indices not materialised 
# !!! 25-06-08: Not actually used in solution 1
def trim_ccr(s: str):
    question_pound_indices = [i for i, char in enumerate(s) if char in {"?", "#"}]
    line_trimmed = s[question_pound_indices[0]:(question_pound_indices[-1]+1)]
    
    return line_trimmed if question_pound_indices else s
pis_ccr = list(map(trim_ccr, pis_ccr))
print(pis_ccr[0:20])

['??????#?#?#??', '#?????????#????', '?#??#..#?#???#??#.', '#??????#???##??#????', '??.?????.?', '?.?.?#?##????.?.?#', '#..?????#.???#?', '#?#??????#??????.', '??????.????#?#??.', '?#?????.?..??.#??.', '??.#?.#???.#???', '###?.?#??.?#?.', '.?...??#???', '????#.?.???#??#?#?.', '.??.?#?????????', '?#.????#????.????#', '?#???.?????###', '?????#.??#', '??????.?##?#??#?????', '.?..??.????#??']
['??????#?#?#??', '#?????????#????', '?#??#..#?#???#??#', '#??????#???##??#????', '??.?????.?', '?.?.?#?##????.?.?#', '#..?????#.???#?', '#?#??????#??????', '??????.????#?#??', '?#?????.?..??.#??', '??.#?.#???.#???', '###?.?#??.?#?', '?...??#???', '????#.?.???#??#?#?', '??.?#?????????', '?#.????#????.????#', '?#???.?????###', '?????#.??#', '??????.?##?#??#?????', '?..??.????#??']


In [64]:
pis_wrs = [[int(num) for num in check.split(",")] for check in pis_wrs]
pis_wrs[0:20]

[[2, 2, 6],
 [1, 3, 2, 2],
 [5, 1, 1, 4],
 [2, 1, 3, 9],
 [3, 1],
 [1, 1, 3, 2, 1],
 [1, 1, 1, 1, 4],
 [1, 1, 8, 1, 1],
 [2, 1, 6],
 [1, 2, 1, 2, 1, 1],
 [1, 2, 1, 1, 4],
 [4, 4, 2],
 [1, 1],
 [2, 2, 3, 5],
 [1, 2, 1, 1, 1],
 [1, 6, 1, 2],
 [2, 1, 3, 4],
 [2, 1, 3],
 [3, 1, 12],
 [1, 1, 3, 1]]

# Problem Setup

# General thoughts

# Solutions

## Solution 1:

#### Remarks and notation

**R - Record:** 
- A record is a line of the line-split input `puzzle_input_split`  
- Example: `?###???????? 3,2,1`  

**CCR - Censored Condition (of the) Record:**  
- The leftmost part of the record containing the (partially) censored condition of the springs.
- From the puzzle input
- Example: `?###????????`  

**WRS - Working Record Specification:**  
- The rightmost part of the record containing the cardinality of each continuous arrangement of working springs (uncensored).  
- From the puzzle input
- Example: `3,2,1`  

**PCR: Possible Condition Record:**  
- A possible uncensoring of the CCR. 
- Generated from the CCR.
- For each R we are looking for the number of (correct) PCRs that have PRSs that correspond to the WRS of the particular R in the input.
- Example: `.###.....##.`

**PRS - Possible (working) Record Specification (of a PCR):** 
- The record specification of a PCR
- Generated from the PCR which is generated from the CCR.
- Example: `3,2` from PCR `.###.....##.`

**KCR - Known (part of the) Condition Record:**
- The uncensored part of a CCR


 

### Explanation of method

### Implementation

#### Part 01:

We create a function to generate an iterator that will generate Cartesian product combinations of `combinants` to generate PCRs.  
The function also takes in a Callable `condition` that might be used to filter amongst the outputted objects of the iterator. We will use this to apply KCR-filters for each R.

In [65]:
# Generate PCRs
def generate_pcr(n: int, combinants: list[str], condition: Callable[[str], bool] = lambda st: True) -> Iterator[str]:
    return (
        ''.join(p) for p in itertools.product(combinants, repeat=n)
        if condition(''.join(p))
    )

In [66]:
# KCR filter function:
def filter_pcr_by_kcr(pcr: str, kcr_filter: dict[int, str]) -> bool:
    return True if all(pcr[i] == kcr_filter[i] for i in kcr_filter) else False

In [67]:
# Let's create a function that generates the kcr filter dictionaries:
def generate_kcr_filter_from_ccr(ccr: str, known_components: set[str] = {".", "#"}) -> dict[int, str]:
    return {i: char for i, char in enumerate(ccr) if char in known_components}

In [68]:
# Let's create a closure function
def generate_kcr_filter_func_from_ccr(ccr: str, known_components: set[str] = {".", "#"}) -> Callable[[str], bool]:
    kcr_filter = generate_kcr_filter_from_ccr(ccr = ccr, known_components=known_components)
    def filter_pcr(pcr: str) -> bool:
        return filter_pcr_by_kcr(pcr = pcr, kcr_filter=kcr_filter)
    return filter_pcr

#### Part 02:

We generate prs from pcrs

In [69]:
def generate_prs_from_pcr(pcr: str) -> list[int]:
    return [len(component) for component in pcr.split(".") if component]

#### Part 03:

!!! 25-06-09 skal vi have parametrisering på basis af R, eller med input fra både ccr og wrs

We run over the puzzle_input

In [70]:
def calc_num_pcr_from_ccr(ccr: str, wrs:list[int]) -> int:
    filter_func = generate_kcr_filter_func_from_ccr(ccr = ccr)
    len_ccr = len(ccr)
    desired_combinants = ["#", "."]
    return sum(
        1
        for pcr in generate_pcr(n=len_ccr, combinants=desired_combinants, condition=filter_func)
        if generate_prs_from_pcr(pcr=pcr) == wrs
    )

In [71]:
def show_num_pcr_from_ccr(ccr: str, wrs: list[int]) -> None:
    i = 0
    filter_func = generate_kcr_filter_func_from_ccr(ccr = ccr)
    print(f"ccr: {ccr}, wrs: {wrs}\n")
    for pcr in generate_pcr(len(ccr), ["#", "."], filter_func):
        prs = generate_prs_from_pcr(pcr)
        if prs == wrs:
            print(f"pcr: {pcr}, prs: {prs}")
            i += 1
    print(f"\ntotal number of pcrs: {i}\n")

In [72]:
for k in range(0,10):
    show_num_pcr_from_ccr(pis_ccr[k], pis_wrs[k])

ccr: ??????#?#?#??, wrs: [2, 2, 6]

pcr: ##.##.######., prs: [2, 2, 6]

total number of pcrs: 1

ccr: #?????????#????, wrs: [1, 3, 2, 2]

pcr: #.###.##.##...., prs: [1, 3, 2, 2]
pcr: #.###.##..##..., prs: [1, 3, 2, 2]
pcr: #.###..##.##..., prs: [1, 3, 2, 2]
pcr: #.###....##.##., prs: [1, 3, 2, 2]
pcr: #.###....##..##, prs: [1, 3, 2, 2]
pcr: #.###.....##.##, prs: [1, 3, 2, 2]
pcr: #..###.##.##..., prs: [1, 3, 2, 2]
pcr: #..###...##.##., prs: [1, 3, 2, 2]
pcr: #..###...##..##, prs: [1, 3, 2, 2]
pcr: #..###....##.##, prs: [1, 3, 2, 2]
pcr: #...###..##.##., prs: [1, 3, 2, 2]
pcr: #...###..##..##, prs: [1, 3, 2, 2]
pcr: #...###...##.##, prs: [1, 3, 2, 2]
pcr: #....###.##.##., prs: [1, 3, 2, 2]
pcr: #....###.##..##, prs: [1, 3, 2, 2]
pcr: #....###..##.##, prs: [1, 3, 2, 2]
pcr: #.....###.##.##, prs: [1, 3, 2, 2]

total number of pcrs: 17

ccr: ?#??#..#?#???#??#, wrs: [5, 1, 1, 4]

pcr: #####..#.#...####, prs: [5, 1, 1, 4]

total number of pcrs: 1

ccr: #??????#???##??#????, wrs: [2, 1, 3, 9]

In [77]:
nums = [calc_num_pcr_from_ccr(ccr, wrs) for ccr, wrs in zip(pis_ccr, pis_wrs)]

In [80]:
sum(nums)

7490

### Checking runtime

#### Benchmarking

#### Runtime profiling

## Solution 2:

# Experimental

## Solution 1

In [128]:
pis_ccr

['??????#?#?#??',
 '#?????????#????',
 '?#??#..#?#???#??#.',
 '#??????#???##??#????',
 '??.?????.?',
 '?.?.?#?##????.?.?#',
 '#..?????#.???#?',
 '#?#??????#??????.',
 '??????.????#?#??.',
 '?#?????.?..??.#??.',
 '??.#?.#???.#???',
 '###?.?#??.?#?.',
 '.?...??#???',
 '????#.?.???#??#?#?.',
 '.??.?#?????????',
 '?#.????#????.????#',
 '?#???.?????###',
 '?????#.??#',
 '??????.?##?#??#?????',
 '.?..??.????#??',
 '#.?#?#?..?????.??..',
 '.???#?.?#?#??',
 '???.??#??.??',
 '?????##?#??????????.',
 '..??#?????..?##',
 '????.#??.#',
 '????###.???#??.#.',
 '??#?.#??.?#?.???',
 '.#???#?#...###?##?#',
 '.?#??.????',
 '#?#?.???#????####',
 '#?#?.?????##??#????',
 '?####?????.#???',
 '??.??.???##??',
 '#?#??#??..#??##??',
 '????????????',
 '#?????#???.????',
 '?????.????#?#?',
 '?.##???.?#????',
 '?.??##????????.',
 '??#?##???##?????#?',
 '?????.#.???#??.?',
 '?????????..',
 '..#??#?????.#',
 '???.#?.?????',
 '?..?#..??#?.',
 '??#????????.???####',
 '??###?#????##?.',
 '.??#??#???.#????#',
 '#??#?.?

In [129]:
# Let's make a function for trimming the beginning and end of lines for "."s
# we use "in {}" speedup as benchmarked to be faster than "== or ==" or "in []"
# !!! 25-06-08: Should be able to speed up more by having indices not materialised 
def trim_ccr(line: str):
    question_pound_indices = [i for i, char in enumerate(line) if char in {"?", "#"}]
    line_trimmed = line[question_pound_indices[0]:(question_pound_indices[-1]+1)]
    
    return line_trimmed if not question_pound_indices else line
    

In [130]:
test_line = "..#.?#?#?..?????.??.."

In [131]:
trim_ccr(test_line)

'..#.?#?#?..?????.??..'

In [132]:
# Let's trim ccr
pis_ccr = list(map(trim_ccr, pis_ccr))

In [133]:
pis_ccr

['??????#?#?#??',
 '#?????????#????',
 '?#??#..#?#???#??#.',
 '#??????#???##??#????',
 '??.?????.?',
 '?.?.?#?##????.?.?#',
 '#..?????#.???#?',
 '#?#??????#??????.',
 '??????.????#?#??.',
 '?#?????.?..??.#??.',
 '??.#?.#???.#???',
 '###?.?#??.?#?.',
 '.?...??#???',
 '????#.?.???#??#?#?.',
 '.??.?#?????????',
 '?#.????#????.????#',
 '?#???.?????###',
 '?????#.??#',
 '??????.?##?#??#?????',
 '.?..??.????#??',
 '#.?#?#?..?????.??..',
 '.???#?.?#?#??',
 '???.??#??.??',
 '?????##?#??????????.',
 '..??#?????..?##',
 '????.#??.#',
 '????###.???#??.#.',
 '??#?.#??.?#?.???',
 '.#???#?#...###?##?#',
 '.?#??.????',
 '#?#?.???#????####',
 '#?#?.?????##??#????',
 '?####?????.#???',
 '??.??.???##??',
 '#?#??#??..#??##??',
 '????????????',
 '#?????#???.????',
 '?????.????#?#?',
 '?.##???.?#????',
 '?.??##????????.',
 '??#?##???##?????#?',
 '?????.#.???#??.?',
 '?????????..',
 '..#??#?????.#',
 '???.#?.?????',
 '?..?#..??#?.',
 '??#????????.???####',
 '??###?#????##?.',
 '.??#??#???.#????#',
 '#??#?.?

In [134]:
def split_full_stop(line: str) -> list:
    return line.split(".")

In [135]:
split_full_stop

<function __main__.split_full_stop(line: str) -> list>

In [136]:
# let's make a function that generates a list of the number of continuous "#" substrings and their lengths.
def count_pound(line: str) -> list[int]:
    line.split()

In [137]:
puzzle_input_split

['??????#?#?#?? 2,2,6',
 '#?????????#???? 1,3,2,2',
 '?#??#..#?#???#??#. 5,1,1,4',
 '#??????#???##??#???? 2,1,3,9',
 '??.?????.? 3,1',
 '?.?.?#?##????.?.?# 1,1,3,2,1',
 '#..?????#.???#? 1,1,1,1,4',
 '#?#??????#??????. 1,1,8,1,1',
 '??????.????#?#??. 2,1,6',
 '?#?????.?..??.#??. 1,2,1,2,1,1',
 '??.#?.#???.#??? 1,2,1,1,4',
 '###?.?#??.?#?. 4,4,2',
 '.?...??#??? 1,1',
 '????#.?.???#??#?#?. 2,2,3,5',
 '.??.?#????????? 1,2,1,1,1',
 '?#.????#????.????# 1,6,1,2',
 '?#???.?????### 2,1,3,4',
 '?????#.??# 2,1,3',
 '??????.?##?#??#????? 3,1,12',
 '.?..??.????#?? 1,1,3,1',
 '#.?#?#?..?????.??.. 1,5,5,1',
 '.???#?.?#?#?? 1,1,4',
 '???.??#??.?? 3,1',
 '?????##?#??????????. 1,6,2,1,1',
 '..??#?????..?## 4,3',
 '????.#??.# 1,1,1',
 '????###.???#??.#. 7,5,1',
 '??#?.#??.?#?.??? 2,2,2,1,1',
 '.#???#?#...###?##?# 1,4,8',
 '.?#??.???? 4,1',
 '#?#?.???#????#### 1,1,4,6',
 '#?#?.?????##??#???? 1,2,8,1,1',
 '?####?????.#??? 5,1,3',
 '??.??.???##?? 1,1,7',
 '#?#??#??..#??##?? 1,6,1,3',
 '???????????? 1,1,1,4'

In [138]:
import itertools
from typing import Callable, Iterator

def generate_combinations(n: int, combinants: list[str], condition: Callable[[str], bool] = lambda x: True) -> Iterator[str]:
    combinants_str = "".join(combinants)
    return (
        ''.join(p) for p in itertools.product(combinants, repeat=n)
        if condition(''.join(p))
    )

In [139]:
# Only combinations ending in "bb"
def ends_with_bb(s: str) -> bool:
    return s.endswith('bb')

for combo in generate_combinations(3, ["a", "b"], ends_with_bb):
    print(combo)

abb
bbb


In [140]:
for combo in generate_combinations(3, ["a", "b"]):
    print(combo)

aaa
aab
aba
abb
baa
bab
bba
bbb


In [141]:
test_list = ["a", "b"]

In [142]:
print("".join(test_list))

ab


### test set experimentation

In [143]:
test_pis_cens_cond_rec = [
    "???.###",
    ".??..??...?##.",
    "?#?#?#?#?#?#?#?",
    "????.#...#...",
    "????.######..#####.",
    "?###????????",
    ]

In [144]:
test_pis_check = [
 "1,1,3",
 "1,1,3",
 "1,3,1,6",
 "4,1,1",
 "1,6,5",
 "3,2,1",
]

In [145]:
test_pis_correct_combinations = [1,4,1,1,4,10]

In [ ]:
# let's try to generate all combinations that will satisfy the game conditions for a single example
# censored condition record
test_case_ccr = "?###????????"

# working records specification
test_case_wrs = [3,2,1]

In [147]:
def test_filter_index(s: str, test_case_dict: dict) -> bool:
    # True if all(s[i] == test_case_dict[i] for i, char in enumerate(s) if char in ["#", "."]) else False
    # return True if all(char == test_case_dict[i] for i, char in enumerate(s) if char in ["#", "."]) else False
    return True if all(s[i] == test_case_dict[i] for i in test_case_dict) else False

In [155]:
i = 0
#possible condition record
for pcr in generate_combinations(len(test_case_ccr), [".", "#"]):
    print(pcr)
    i+=1
    if i > 20:
        break

............
...........#
..........#.
..........##
.........#..
.........#.#
.........##.
.........###
........#...
........#..#
........#.#.
........#.##
........##..
........##.#
........###.
........####
.......#....
.......#...#
.......#..#.
.......#..##
.......#.#..


In [156]:
from functools import partial
test_case_dict = {1: "#", 2: "#", 3: "#"}
test_case_filter_index = partial(test_filter_index, test_case_dict = test_case_dict)
i = 0
for pcr in generate_combinations(len(test_case_ccr), [".", "#"], test_case_filter_index):
    print(pcr)
    i += 1
print(i)

.###........
.###.......#
.###......#.
.###......##
.###.....#..
.###.....#.#
.###.....##.
.###.....###
.###....#...
.###....#..#
.###....#.#.
.###....#.##
.###....##..
.###....##.#
.###....###.
.###....####
.###...#....
.###...#...#
.###...#..#.
.###...#..##
.###...#.#..
.###...#.#.#
.###...#.##.
.###...#.###
.###...##...
.###...##..#
.###...##.#.
.###...##.##
.###...###..
.###...###.#
.###...####.
.###...#####
.###..#.....
.###..#....#
.###..#...#.
.###..#...##
.###..#..#..
.###..#..#.#
.###..#..##.
.###..#..###
.###..#.#...
.###..#.#..#
.###..#.#.#.
.###..#.#.##
.###..#.##..
.###..#.##.#
.###..#.###.
.###..#.####
.###..##....
.###..##...#
.###..##..#.
.###..##..##
.###..##.#..
.###..##.#.#
.###..##.##.
.###..##.###
.###..###...
.###..###..#
.###..###.#.
.###..###.##
.###..####..
.###..####.#
.###..#####.
.###..######
.###.#......
.###.#.....#
.###.#....#.
.###.#....##
.###.#...#..
.###.#...#.#
.###.#...##.
.###.#...###
.###.#..#...
.###.#..#..#
.###.#..#.#.
.###.#..#.##
.###.#..##..

In [150]:
# Let's create a function that generates the filter dictionaries:
def generate_filter_dict(s: str, filter: set[str]) -> dict:
    return {i: char for i, char in enumerate(s) if char in filter}

In [151]:
generate_filter_dict(test_case_ccr, {"#", "."})

{1: '#', 2: '#', 3: '#'}

Having had the filter for the uncensored part of the censored condition records (CCR), we need to implement a filter for the working records specification (WRS).
We might do this smart somehow, but a dumb way is to simply check each of the possible outputs and whether they match the working records specification (WRS) for that record (take intersection).
To do that we might make a function to generate a record specification (RS) from a "possible condition record" (PCR).

In [152]:
def generate_rs_from_pcr(s: str) -> list[int]:
    return [len(component) for component in s.split(".") if component]

In [153]:
test_case_pcr = ".###....###."
generate_rs_from_pcr(test_case_pcr)

[3, 3]

In [160]:
i = 0
print(f"ccr: {test_case_ccr}, wrs: {test_case_wrs}")
for pcr in generate_combinations(len(test_case_ccr), [".", "#"], test_case_filter_index):
    print(f"pcr: {pcr}, rs: {generate_rs_from_pcr(pcr)}")
    i += 1
print(f"total number of pcrs: {i}")

ccr: ?###????????, wrs: [3, 2, 1]
pcr: .###........, rs: [3]
pcr: .###.......#, rs: [3, 1]
pcr: .###......#., rs: [3, 1]
pcr: .###......##, rs: [3, 2]
pcr: .###.....#.., rs: [3, 1]
pcr: .###.....#.#, rs: [3, 1, 1]
pcr: .###.....##., rs: [3, 2]
pcr: .###.....###, rs: [3, 3]
pcr: .###....#..., rs: [3, 1]
pcr: .###....#..#, rs: [3, 1, 1]
pcr: .###....#.#., rs: [3, 1, 1]
pcr: .###....#.##, rs: [3, 1, 2]
pcr: .###....##.., rs: [3, 2]
pcr: .###....##.#, rs: [3, 2, 1]
pcr: .###....###., rs: [3, 3]
pcr: .###....####, rs: [3, 4]
pcr: .###...#...., rs: [3, 1]
pcr: .###...#...#, rs: [3, 1, 1]
pcr: .###...#..#., rs: [3, 1, 1]
pcr: .###...#..##, rs: [3, 1, 2]
pcr: .###...#.#.., rs: [3, 1, 1]
pcr: .###...#.#.#, rs: [3, 1, 1, 1]
pcr: .###...#.##., rs: [3, 1, 2]
pcr: .###...#.###, rs: [3, 1, 3]
pcr: .###...##..., rs: [3, 2]
pcr: .###...##..#, rs: [3, 2, 1]
pcr: .###...##.#., rs: [3, 2, 1]
pcr: .###...##.##, rs: [3, 2, 2]
pcr: .###...###.., rs: [3, 3]
pcr: .###...###.#, rs: [3, 3, 1]
pcr: .###...####.,

We see that we might skip any PCRs thta don't have 3 as their first working record number:

In [164]:
i = 0
print(f"ccr: {test_case_ccr}, wrs: {test_case_wrs}\n")
for pcr in generate_combinations(len(test_case_ccr), [".", "#"], test_case_filter_index):
    rs = generate_rs_from_pcr(pcr)
    if rs[0] != test_case_wrs[0]:
        continue
    else:
        print(f"pcr: {pcr}, rs: {rs}")
        i += 1
print(f"\ntotal number of pcrs: {i}")

ccr: ?###????????, wrs: [3, 2, 1]

pcr: .###........, rs: [3]
pcr: .###.......#, rs: [3, 1]
pcr: .###......#., rs: [3, 1]
pcr: .###......##, rs: [3, 2]
pcr: .###.....#.., rs: [3, 1]
pcr: .###.....#.#, rs: [3, 1, 1]
pcr: .###.....##., rs: [3, 2]
pcr: .###.....###, rs: [3, 3]
pcr: .###....#..., rs: [3, 1]
pcr: .###....#..#, rs: [3, 1, 1]
pcr: .###....#.#., rs: [3, 1, 1]
pcr: .###....#.##, rs: [3, 1, 2]
pcr: .###....##.., rs: [3, 2]
pcr: .###....##.#, rs: [3, 2, 1]
pcr: .###....###., rs: [3, 3]
pcr: .###....####, rs: [3, 4]
pcr: .###...#...., rs: [3, 1]
pcr: .###...#...#, rs: [3, 1, 1]
pcr: .###...#..#., rs: [3, 1, 1]
pcr: .###...#..##, rs: [3, 1, 2]
pcr: .###...#.#.., rs: [3, 1, 1]
pcr: .###...#.#.#, rs: [3, 1, 1, 1]
pcr: .###...#.##., rs: [3, 1, 2]
pcr: .###...#.###, rs: [3, 1, 3]
pcr: .###...##..., rs: [3, 2]
pcr: .###...##..#, rs: [3, 2, 1]
pcr: .###...##.#., rs: [3, 2, 1]
pcr: .###...##.##, rs: [3, 2, 2]
pcr: .###...###.., rs: [3, 3]
pcr: .###...###.#, rs: [3, 3, 1]
pcr: .###...####.

In [ ]:
# We see that we can find a solution for the test_case
i = 0
print(f"ccr: {test_case_ccr}, wrs: {test_case_wrs}\n")
for pcr in generate_combinations(len(test_case_ccr), [".", "#"], test_case_filter_index):
    rs = generate_rs_from_pcr(pcr)
    if rs != test_case_wrs:
        continue
    else:
        print(f"pcr: {pcr}, rs: {rs}")
        i += 1
print(f"\ntotal number of pcrs: {i}")

ccr: ?###????????, wrs: [3, 2, 1]

pcr: .###....##.#, rs: [3, 2, 1]
pcr: .###...##..#, rs: [3, 2, 1]
pcr: .###...##.#., rs: [3, 2, 1]
pcr: .###..##...#, rs: [3, 2, 1]
pcr: .###..##..#., rs: [3, 2, 1]
pcr: .###..##.#.., rs: [3, 2, 1]
pcr: .###.##....#, rs: [3, 2, 1]
pcr: .###.##...#., rs: [3, 2, 1]
pcr: .###.##..#.., rs: [3, 2, 1]
pcr: .###.##.#..., rs: [3, 2, 1]

total number of pcrs: 10


We see that we have an approach that will work, but is crude. This will be our solution 1.   
For solution 2 we might try to optimise solution 1.  
In a solution 3, one could optimise the running through of the possibilities by calculating proper PCRs.